In [31]:
# model fit params
gbm_params = {
    'n_estimators' : 500, # for accuracy use large numbers like 6000 
    'learning_rate': 0.4,
    'feature_fraction' : 0.9,
    'subsample' : 0.1,  # 
    'subsample_freq' : 1,
    'num_leaves' : 20,
    'max_depth' : 10,
    'metric':'rmse',
    'lambda_l1' : 1,  
    'lambda_l2': 1,
    'verbose': 100
}

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
import gc
from os import path
from sklearn.preprocessing import LabelEncoder
from pandas.core.dtypes.dtypes import CategoricalDtype
from tqdm import tqdm
from datetime import date 
import holidays
import lightgbm as lgb


warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

# label encoding
le = LabelEncoder()

In [2]:
class ConvertToDatetime(TransformerMixin):
        
    def transform(self, df, **transform_params):
        if 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [3]:
file_dtype = {
    'train': {'building_id': np.int16, 'meter': np.int8, 'meter_reading': np.float32},
    'test': {'building_id': np.int16, 'meter': np.int8},
    'building_metadata': {'site_id': np.int8, 'building_id': np.uint16, 'square_feet': np.float32, 'year_built': np.float16, 'floor_count': np.float16},
    'weather_test' : {'site_id': np.int8, 'air_temperature': np.float16, 'cloud_coverage': np.float16, 'dew_temperature': np.float16,
                     'precip_depth_1_hr': np.float16, 'sea_level_pressure': np.float16, 'wind_direction': np.float16, 'wind_speed': np.float16},
    'weather_train' : {'site_id': np.int8, 'air_temperature': np.float16, 'cloud_coverage': np.float16, 'dew_temperature': np.float16,
                     'precip_depth_1_hr': np.float16, 'sea_level_pressure': np.float16, 'wind_direction': np.float16, 'wind_speed': np.float16}
}

def loadFile(name):
    for dir_path in ['../input/ashrae-energy-prediction/','../input/_ashrae-energy-prediction/']:
        if path.exists(dir_path + name + '.csv'):
            return  ConvertToDatetime().transform(
                pd.read_csv(dir_path + name + '.csv', dtype=file_dtype[name]))
        


In [4]:
building = loadFile('building_metadata')
weather_train = loadFile('weather_train')
weather_test = loadFile('weather_test')
train = loadFile('train')
test = loadFile('test')



In [5]:
# align weather
def weatherSiteOffsets():
    weather = pd.concat([weather_train,weather_test],ignore_index=True)
    weather['timestamp'] = pd.to_datetime(weather['timestamp'])
    weather_key = ['site_id', 'timestamp']

    temp_skeleton = weather[weather_key + ['air_temperature']].drop_duplicates(subset=weather_key).sort_values(by=weather_key).copy()

    # calculate ranks of hourly temperatures within date/site_id chunks
    temp_skeleton['temp_rank'] = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.date])['air_temperature'].rank('average')
    
    # create a dataframe of site_ids (0-16) x mean hour rank of temperature within day (0-23)
    df_2d = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.hour])['temp_rank'].mean().unstack(level=1)

    # Subtract the columnID of temperature peak by 14, getting the timestamp alignment gap.
    site_ids_offsets = pd.Series(df_2d.values.argmax(axis=1) - 14)
    site_ids_offsets.index.name = 'site_id'
    return site_ids_offsets

site_time_offsets_df = weatherSiteOffsets()

def alignWeather(df):
        df['offset'] = df.site_id.map(site_time_offsets_df)
        df['timestamp_aligned'] = (df.timestamp - pd.to_timedelta(df.offset, unit='H'))
        df['timestamp'] = df['timestamp_aligned']
        del df['timestamp_aligned'], df['offset']
        gc.collect()
        return df

weather_train = alignWeather(weather_train)
weather_test = alignWeather(weather_test)

del site_time_offsets_df

In [6]:
# See holiday notebook to generate
holiday_df = pd.read_pickle('../input/ashrae-energy-prediction/holiday_df.pickle')

In [7]:
# "As you can see above, this data looks weired until May 20. It is 
# reported in this discussion by @barnwellguy that All electricity
# meter is 0 until May 20 for site_id == 0. Let's remove these data 
# from training data."
# https://www.kaggle.com/kaushal2896/ashrae-eda-fe-lightgbm-1-13
class RmS0M0(TransformerMixin):
  
    def transform(self, df, **transform_params):
        return df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

    def fit(self, X, y=None, **fit_params):
        return self
    


In [8]:
class ImputeWeather(TransformerMixin):

    def __init__(self, method:str='linear', gap_limit:int=None, limit_direction:str='forward'):
        self._method = method
        self._gap_limit = gap_limit
        self._limit_direction = limit_direction
        
    def transform(self, weather_df, **transform_params):
        grouped_weather_df = weather_df.groupby('site_id').apply(lambda group: group.interpolate(method=self._method, limit=self._gap_limit, limit_direction=self._limit_direction))
        if 'cloud_coverage' in grouped_weather_df.columns:
            grouped_weather_df['cloud_coverage'] = grouped_weather_df['cloud_coverage'].round(decimals=0).clip(0,8)
        grouped_weather_df.reset_index(inplace=True)
        weather_df = grouped_weather_df.drop(['index'], axis=1)
        gc.collect()
        return weather_df

    def fit(self, X, y=None, **fit_params):
        return self
print(weather_train.head(20))
print(ImputeWeather().transform(weather_train.head(20)))

    site_id           timestamp  air_temperature  cloud_coverage  \
0         0 2015-12-31 19:00:00        25.000000             6.0   
1         0 2015-12-31 20:00:00        24.406250             NaN   
2         0 2015-12-31 21:00:00        22.796875             2.0   
3         0 2015-12-31 22:00:00        21.093750             2.0   
4         0 2015-12-31 23:00:00        20.000000             2.0   
5         0 2016-01-01 00:00:00        19.406250             NaN   
6         0 2016-01-01 01:00:00        21.093750             6.0   
7         0 2016-01-01 02:00:00        21.093750             NaN   
8         0 2016-01-01 03:00:00        20.593750             NaN   
9         0 2016-01-01 04:00:00        21.093750             NaN   
10        0 2016-01-01 05:00:00        21.093750             NaN   
11        0 2016-01-01 06:00:00        20.593750             NaN   
12        0 2016-01-01 07:00:00        18.906250             6.0   
13        0 2016-01-01 08:00:00        20.000000

In [9]:
class AddWeatherLags(TransformerMixin):
    
    def __init__(self, window):
        self._window = window
        
    def transform(self, weather_df, **transform_params):
        group_df = weather_df.groupby(['site_id'])
        cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
        rolled = group_df[cols].rolling(window=self._window, min_periods=0)
        lag_mean = rolled.mean().reset_index().astype(np.float16)
        lag_max = rolled.max().reset_index().astype(np.float16)
        lag_min = rolled.min().reset_index().astype(np.float16)
        lag_std = rolled.std().reset_index().astype(np.float16)
        for col in cols:
            weather_df[f'{col}_mean_lag{self._window}'] = lag_mean[col]
            weather_df[f'{col}_max_lag{self._window}'] = lag_max[col]
            weather_df[f'{col}_min_lag{self._window}'] = lag_min[col]
            weather_df[f'{col}_std_lag{self._window}'] = lag_std[col]
        del group_df, rolled
        gc.collect()
        return weather_df

    def fit(self, X, y=None, **fit_params):
        return self
    
print(AddWeatherLags(72).transform(weather_train.head(20)))

    site_id           timestamp  air_temperature  cloud_coverage  \
0         0 2015-12-31 19:00:00        25.000000             6.0   
1         0 2015-12-31 20:00:00        24.406250             NaN   
2         0 2015-12-31 21:00:00        22.796875             2.0   
3         0 2015-12-31 22:00:00        21.093750             2.0   
4         0 2015-12-31 23:00:00        20.000000             2.0   
5         0 2016-01-01 00:00:00        19.406250             NaN   
6         0 2016-01-01 01:00:00        21.093750             6.0   
7         0 2016-01-01 02:00:00        21.093750             NaN   
8         0 2016-01-01 03:00:00        20.593750             NaN   
9         0 2016-01-01 04:00:00        21.093750             NaN   
10        0 2016-01-01 05:00:00        21.093750             NaN   
11        0 2016-01-01 06:00:00        20.593750             NaN   
12        0 2016-01-01 07:00:00        18.906250             6.0   
13        0 2016-01-01 08:00:00        20.000000

[20 rows x 37 columns]


In [10]:
class AddBuilding(TransformerMixin):

    def __init__(self, building_df):
        self._b_df = building_df
        
    def transform(self, df, **transform_params):
        return df.merge(_b_df, on='building_id', how='left')

    def fit(self, X, y=None, **fit_params):
        return self

In [11]:
    
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114483#latest-660771
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114874#latest-660970
class AddHolidays(TransformerMixin):
    def transform(self, df, **transform_params):
        df = df.merge(holiday_df, on=['building_id','meter','timestamp','site_id'], how='left')
        return df

    def fit(self, X, y=None, **fit_params):
        return self
# Test 
addHolidays = AddHolidays()
print(addHolidays.transform(train.head(2000).merge(building, on='building_id', how='left'))[['building_id','timestamp','holiday']])

      building_id  timestamp         holiday
0               0 2016-01-01  New Year's Day
1               1 2016-01-01  New Year's Day
2               2 2016-01-01  New Year's Day
3               3 2016-01-01  New Year's Day
4               4 2016-01-01  New Year's Day
5               5 2016-01-01  New Year's Day
6               6 2016-01-01  New Year's Day
7               7 2016-01-01  New Year's Day
8               8 2016-01-01  New Year's Day
9               9 2016-01-01  New Year's Day
10             10 2016-01-01  New Year's Day
11             11 2016-01-01  New Year's Day
12             12 2016-01-01  New Year's Day
13             13 2016-01-01  New Year's Day
14             14 2016-01-01  New Year's Day
15             15 2016-01-01  New Year's Day
16             16 2016-01-01  New Year's Day
17             17 2016-01-01  New Year's Day
18             18 2016-01-01  New Year's Day
19             19 2016-01-01  New Year's Day
20             20 2016-01-01  New Year's Day
21        

In [12]:
class RmHolidays(TransformerMixin):
    def transform(self, df, **transform_params):
        df = df.merge(holiday_df, on=['building_id','meter','timestamp','site_id'], how='left')
        df = df.drop(df[df['holiday'].notnull()].index)
        df = df.drop(['holiday'], axis=1)
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

# Test you should see the new years removed
rmHolidays = RmHolidays()
print(rmHolidays.transform(train.head(100000).merge(building, on='building_id', how='left')))

       building_id  meter           timestamp  meter_reading  site_id  \
55121            0      0 2016-01-02 00:00:00       0.000000        0   
55122            1      0 2016-01-02 00:00:00       0.000000        0   
55123            2      0 2016-01-02 00:00:00       0.000000        0   
55124            3      0 2016-01-02 00:00:00       0.000000        0   
55125            4      0 2016-01-02 00:00:00       0.000000        0   
55126            5      0 2016-01-02 00:00:00       0.000000        0   
55127            6      0 2016-01-02 00:00:00       0.000000        0   
55128            7      0 2016-01-02 00:00:00       0.000000        0   
55129            8      0 2016-01-02 00:00:00       0.000000        0   
55130            9      0 2016-01-02 00:00:00       0.000000        0   
55131           10      0 2016-01-02 00:00:00       0.000000        0   
55132           11      0 2016-01-02 00:00:00       0.000000        0   
55133           12      0 2016-01-02 00:00:00      

[41839 rows x 9 columns]


In [13]:
class LogSquareFeet(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['log_square_feet'] = np.float16(np.log(df['square_feet']))
        return df

    def fit(self, X, y=None, **fit_params):
        return self
print(building.head(20)['square_feet'])

0       7432.0
1       2720.0
2       5376.0
3      23685.0
4     116607.0
5       8000.0
6      27926.0
7     121074.0
8      60809.0
9      27000.0
10    370773.0
11     49073.0
12     37100.0
13     99380.0
14     86250.0
15     83957.0
16     54644.0
17     15250.0
18    111891.0
19     18717.0
Name: square_feet, dtype: float32


In [14]:
class SetCatTypes(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['primary_use']= df['primary_use'].astype('category')
        df['meter'] = df["meter"].astype('category')
        df['site_id'] = df["site_id"].astype('category')
        df['building_id'] = df['building_id'].astype('category')
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [15]:
class ImputeCloudCoverage(TransformerMixin):
        
    def transform(self, df, **transform_params):
        # set age of building to mediam of site_id
        # else if set ot overall median
        median = df['cloud_coverage'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in df.groupby(['site_id'])['cloud_coverage'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['cloud_coverage'].isnull()) & (df['site_id'] == i), 'cloud_coverage'] = i_median
            else:
                df.loc[(df['cloud_coverage'].isnull()) & (df['site_id'] == i), 'cloud_coverage'] = median
        df['cloud_coverage'] = np.uint8(df['cloud_coverage'])
        df['cloud_coverage'] = df['cloud_coverage']
        del median
        gc.collect()
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self


In [16]:
class CloudTimeCat(TransformerMixin):
        
    def transform(self, df, **transform_params):
        tempDf = df[['cloud_coverage', 'hour']].astype('int')
        tempDf['cloud_coverage'] = (tempDf['cloud_coverage']).astype('int')
        tempDf['hour'] = (tempDf['hour']).astype('int')
        tempDf = tempDf.astype('str')
        df['cloud_time_cat'] = 'c' + tempDf['cloud_coverage'] + 't' + tempDf['hour']
        df['cloud_time_cat'] = df['cloud_time_cat'].astype('category')
        del tempDf
        gc.collect()
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self


In [17]:
class DropCols(TransformerMixin):

    def __init__(self, drop_cols):
        self._drop_cols = drop_cols
        
    def transform(self, df, **transform_params):
        df = df.drop(self._drop_cols, axis=1)
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [18]:
class ImputeYearBuilt(TransformerMixin):

    def transform(self, df, **transform_params):
        year_built_median = df['year_built'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in df.groupby(['site_id'])['year_built'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = i_median
            else:
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = year_built_median
        df['building_age'] = np.uint8(df['year_built']-1900)
        del year_built_median
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self


In [19]:
class AddMeterDummies(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        for i in range(4):
            df['_meter_'+str(i)] = (df['building_id'].isin(
                train.loc[train['meter'] == i].building_id.unique()))
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [20]:
class AddTimeFeatures(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        df['dayofweek'] = df['timestamp'].dt.dayofweek.astype('category') # vs weekend?
        #df['weekday'] = df['timestamp'].dt.weekday.astype('category')
        #df['dayofweek_hour'] = (df['timestamp'].dt.dayofweek * 24) + df['timestamp'].dt.hour
        #df['dayofweek_hour'] = df['dayofweek_hour'].astype('category')
        #df['week'] = df['timestamp'].dt.week.astype('category')
        df['hour'] = df['timestamp'].dt.hour.astype('category')
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [21]:
class AddRelativeHumidity(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        # code here
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [22]:
class FillMean(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].mean())
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [23]:
class FillZeros(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(0)
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [24]:
class FillMedian(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].median())
        return df

    def fit(self, X, y=None, **fit_params):
        return self


In [25]:
class FillPopular(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].value_counts()[0])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [26]:
class MarkNaNs(TransformerMixin):
        
    def transform(self, df, **transform_params):
        for col in  df.columns[df.isna().any()].tolist():
            df['_' + col + '_nan' ] = df[col].isnull()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [27]:
class GC(TransformerMixin):
        
    def transform(self, df, **transform_params):
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [28]:
# declare model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_log_error, mean_squared_error
from lightgbm import LGBMRegressor


def rmsle(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(y, y_pred.clip(0)))

def rmse(y, y_pred):
    # hack to prevent negative numbers
    return mean_squared_error(y, y_pred.clip(0))

def rmsee(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(np.expm1(y.clip(0)), np.expm1(y_pred.clip(0))))
    
rmsle_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmse_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmsee_scorer = make_scorer(
    lambda y_true, y_pred : rmsee(y_true, y_pred), 
    greater_is_better=False)

def lbm_rmsle(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False

# rob's custome function to do RMSLE while in the log1p space
def lbm_rmslee(y_true, y_pred):
    return 'RMSLEE', np.sqrt(np.mean(np.power(y_pred - y_true, 2))), False



In [29]:
weather_pipes = Pipeline(
    steps=[
        #('convertToDatetime', ConvertToDatetime()),
        ('imputeWeather', ImputeWeather()),
        ('fillMean',FillMean(['air_temperature','dew_temperature'
                              , 'precip_depth_1_hr', 'sea_level_pressure'])),
        ('imputeCloudCoverage', ImputeCloudCoverage()),
        ('addWeatherLags3', AddWeatherLags(3)),
        ('addWeatherLags72', AddWeatherLags(72)),
    ]
)

building_pipes = Pipeline(
    steps=[
        ('logSquareFeet', LogSquareFeet()),
        ('imputeYearBuilt', ImputeYearBuilt()),
        ('fillMean',FillMean(['floor_count'])),
        ('dropClos', DropCols(['square_feet', 'year_built'])),
    ]
)


# pre_a_pipes is for preprocessing that doesn't change impute
# values
x_pipes = Pipeline(
    steps=[
        #('markNans',MarkNaNs()),
        #('convertToDatetime', ConvertToDatetime()),
        ('rmS0M0', RmS0M0()),
        #('addHolidays', AddHolidays()),
        #('rmHolidays', RmHolidays()), called manually in fold
        ('addRelativeHumidity',AddRelativeHumidity()),
        ('addTimeFeatures', AddTimeFeatures()),
        ('setCatTypes', SetCatTypes()),
        ('fillMean',FillMean([])),
        ('fillZeros',FillZeros([])),
        ('dropClos', DropCols(['timestamp'])),
        ('GC', GC())
    ]
)

In [40]:
def getOutsideFoldXY(train_index):
    X = train.iloc[train_index].drop('meter_reading', axis=1)
    X_buildings = building[building['building_id'].isin(X['building_id'].unique())]
    X_weather = building[building['building_id'].isin(X['building_id'].unique())]
    X = x_pipes.transform(
        X
            .merge(building_pipes.transform(X_buildings), on='building_id', how='left')
            .merge(weather_train_trans, on=['site_id', 'timestamp'], how='left')
        )
    f_train_y = np.log1p(train.iloc[train_index]['meter_reading'])
    print(X.columns)
    return X,f_train_y



def getInFoldXY(train_index):
    X = train.iloc[train_index]
    X_buildings = building[building['building_id'].isin(X['building_id'].unique())]
    X = X.merge(building_pipes.transform(X_buildings), on='building_id', how='left')
    X_weather = weather_train[
        (weather_train['site_id'].isin(X['site_id'].unique())) 
         & (weather_train['timestamp'].isin(X['timestamp'].unique())) 
    ]
    X = x_pipes.transform(
        #rmHolidays.transform(
            X.merge(weather_pipes.transform(X_weather), how='left'))
    #)
    return X.drop('meter_reading', axis=1),  np.log1p(X['meter_reading'])

sample_train_X, sample_train_y = getInFoldXY(train.sample(n=20,  random_state=42).index)
print(sample_train_X)

   building_id meter site_id                    primary_use  floor_count  \
0         1324     1      14  Entertainment/public assembly     3.740234   
1         1013     0      10                      Education     3.000000   
2          229     1       2                      Education     3.740234   
3          217     3       2                      Education     3.740234   
4         1434     0      15                      Education     3.740234   
5         1047     0      12                Public services     3.740234   
6          911     1       9                      Education     3.740234   
7         1039     0      12                      Education     3.000000   
8          265     0       2                         Office     3.740234   
9          896     0       9                      Education     3.740234   
10         973     0       9                         Office     3.740234   
11         813     0       8                Public services     1.000000   
12         8

[20 rows x 72 columns]


In [41]:
%%time
## cross val mdoels

folds = 5

# this stratified strategy from
# https://www.kaggle.com/isaienkov/lightgbm-fe-1-19/notebook
kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

models = []
best_scores = []
for train_index, val_index in kf.split(train, train['building_id']):
    f_train_X, f_train_y = getInFoldXY(train_index)
    f_val_X, f_val_y = getInFoldXY(val_index)
    gbm = LGBMRegressor(**gbm_params)
    gbm.fit(f_train_X, f_train_y,
        eval_set=[(f_val_X, f_val_y)],
        # https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114722#latest-660848
        # eval_metric=lbm_rmslee,
        early_stopping_rounds=20)
    models.append(gbm)
    #y_pred = gbm.predict(f_val_X, num_iteration=gbm.best_iteration_)
    # eval
    #rmsle_score = lbm_rmslee(f_val_X, y_pred)[1]
    best_scores.append(gbm.best_score_)
    del f_train_X, f_train_y, f_val_X, f_val_y, gbm
    gc.collect()


[1]	valid_0's rmse: 1.79945
Training until validation scores don't improve for 20 rounds
[2]	valid_0's rmse: 1.64054
[3]	valid_0's rmse: 1.49991
[4]	valid_0's rmse: 1.45276
[5]	valid_0's rmse: 1.30169
[6]	valid_0's rmse: 1.24702
[7]	valid_0's rmse: 1.21802
[8]	valid_0's rmse: 1.19772
[9]	valid_0's rmse: 1.14591
[10]	valid_0's rmse: 1.13393
[11]	valid_0's rmse: 1.12274
[12]	valid_0's rmse: 1.10688
[13]	valid_0's rmse: 1.10037
[14]	valid_0's rmse: 1.09454
[15]	valid_0's rmse: 1.09042
[16]	valid_0's rmse: 1.06851
[17]	valid_0's rmse: 1.04011
[18]	valid_0's rmse: 1.03423
[19]	valid_0's rmse: 1.02953
[20]	valid_0's rmse: 1.02664
[21]	valid_0's rmse: 1.01561
[22]	valid_0's rmse: 1.00324
[23]	valid_0's rmse: 0.999411
[24]	valid_0's rmse: 0.997523
[25]	valid_0's rmse: 0.991022
[26]	valid_0's rmse: 0.98812
[27]	valid_0's rmse: 0.986143
[28]	valid_0's rmse: 0.981785
[29]	valid_0's rmse: 0.97791
[30]	valid_0's rmse: 0.974738
[31]	valid_0's rmse: 0.971812
[32]	valid_0's rmse: 0.970102
[33]	valid_0

[30]	valid_0's rmse: 0.994668
[31]	valid_0's rmse: 0.991952
[32]	valid_0's rmse: 0.990365
[33]	valid_0's rmse: 0.979557
[34]	valid_0's rmse: 0.977909
[35]	valid_0's rmse: 0.974374
[36]	valid_0's rmse: 0.972377
[37]	valid_0's rmse: 0.970557
[38]	valid_0's rmse: 0.968729
[39]	valid_0's rmse: 0.967738
[40]	valid_0's rmse: 0.966261
[41]	valid_0's rmse: 0.965213
[42]	valid_0's rmse: 0.963549
[43]	valid_0's rmse: 0.962423
[44]	valid_0's rmse: 0.960993
[45]	valid_0's rmse: 0.95715
[46]	valid_0's rmse: 0.956321
[47]	valid_0's rmse: 0.95348
[48]	valid_0's rmse: 0.95202
[49]	valid_0's rmse: 0.949878
[50]	valid_0's rmse: 0.948889
[51]	valid_0's rmse: 0.948362
[52]	valid_0's rmse: 0.943153
[53]	valid_0's rmse: 0.941541
[54]	valid_0's rmse: 0.940379
[55]	valid_0's rmse: 0.939377
[56]	valid_0's rmse: 0.938428
[57]	valid_0's rmse: 0.937379
[58]	valid_0's rmse: 0.936536
[59]	valid_0's rmse: 0.933179
[60]	valid_0's rmse: 0.93249
[61]	valid_0's rmse: 0.931591
[62]	valid_0's rmse: 0.931067
[63]	valid_0's

[61]	valid_0's rmse: 0.936921
[62]	valid_0's rmse: 0.935329
[63]	valid_0's rmse: 0.934067
[64]	valid_0's rmse: 0.932782
[65]	valid_0's rmse: 0.931176
[66]	valid_0's rmse: 0.930367
[67]	valid_0's rmse: 0.929683
[68]	valid_0's rmse: 0.928395
[69]	valid_0's rmse: 0.927138
[70]	valid_0's rmse: 0.926263
[71]	valid_0's rmse: 0.925436
[72]	valid_0's rmse: 0.924822
[73]	valid_0's rmse: 0.923951
[74]	valid_0's rmse: 0.922726
[75]	valid_0's rmse: 0.922098
[76]	valid_0's rmse: 0.921338
[77]	valid_0's rmse: 0.920443
[78]	valid_0's rmse: 0.918477
[79]	valid_0's rmse: 0.917374
[80]	valid_0's rmse: 0.916913
[81]	valid_0's rmse: 0.915426
[82]	valid_0's rmse: 0.914718
[83]	valid_0's rmse: 0.91382
[84]	valid_0's rmse: 0.913123
[85]	valid_0's rmse: 0.911817
[86]	valid_0's rmse: 0.911182
[87]	valid_0's rmse: 0.91027
[88]	valid_0's rmse: 0.909481
[89]	valid_0's rmse: 0.908723
[90]	valid_0's rmse: 0.907709
[91]	valid_0's rmse: 0.906765
[92]	valid_0's rmse: 0.906055
[93]	valid_0's rmse: 0.905465
[94]	valid_0

[92]	valid_0's rmse: 0.900732
[93]	valid_0's rmse: 0.899741
[94]	valid_0's rmse: 0.898816
[95]	valid_0's rmse: 0.898267
[96]	valid_0's rmse: 0.897363
[97]	valid_0's rmse: 0.896888
[98]	valid_0's rmse: 0.896073
[99]	valid_0's rmse: 0.895443
[100]	valid_0's rmse: 0.894845
[101]	valid_0's rmse: 0.893495
[102]	valid_0's rmse: 0.892942
[103]	valid_0's rmse: 0.892563
[104]	valid_0's rmse: 0.89183
[105]	valid_0's rmse: 0.891442
[106]	valid_0's rmse: 0.890465
[107]	valid_0's rmse: 0.890052
[108]	valid_0's rmse: 0.88831
[109]	valid_0's rmse: 0.888098
[110]	valid_0's rmse: 0.887761
[111]	valid_0's rmse: 0.886845
[112]	valid_0's rmse: 0.886166
[113]	valid_0's rmse: 0.885413
[114]	valid_0's rmse: 0.884989
[115]	valid_0's rmse: 0.884559
[116]	valid_0's rmse: 0.884176
[117]	valid_0's rmse: 0.882529
[118]	valid_0's rmse: 0.880401
[119]	valid_0's rmse: 0.87994
[120]	valid_0's rmse: 0.879365
[121]	valid_0's rmse: 0.878787
[122]	valid_0's rmse: 0.878517
[123]	valid_0's rmse: 0.878124
[124]	valid_0's rms

[122]	valid_0's rmse: 0.886635
[123]	valid_0's rmse: 0.886144
[124]	valid_0's rmse: 0.885714
[125]	valid_0's rmse: 0.885456
[126]	valid_0's rmse: 0.8848
[127]	valid_0's rmse: 0.883062
[128]	valid_0's rmse: 0.880797
[129]	valid_0's rmse: 0.88064
[130]	valid_0's rmse: 0.880167
[131]	valid_0's rmse: 0.878223
[132]	valid_0's rmse: 0.877854
[133]	valid_0's rmse: 0.877334
[134]	valid_0's rmse: 0.877059
[135]	valid_0's rmse: 0.876621
[136]	valid_0's rmse: 0.874915
[137]	valid_0's rmse: 0.874444
[138]	valid_0's rmse: 0.874147
[139]	valid_0's rmse: 0.873738
[140]	valid_0's rmse: 0.873246
[141]	valid_0's rmse: 0.872959
[142]	valid_0's rmse: 0.87261
[143]	valid_0's rmse: 0.872351
[144]	valid_0's rmse: 0.872067
[145]	valid_0's rmse: 0.871782
[146]	valid_0's rmse: 0.869303
[147]	valid_0's rmse: 0.869149
[148]	valid_0's rmse: 0.86866
[149]	valid_0's rmse: 0.867573
[150]	valid_0's rmse: 0.86701
[151]	valid_0's rmse: 0.865218
[152]	valid_0's rmse: 0.864828
[153]	valid_0's rmse: 0.864551
[154]	valid_0'

In [42]:
# Scores for cross val
for score in best_scores:
    print(score['valid_0']['rmse'])

0.7884182058867528
0.7885826803546043
0.7892835061874126
0.7839556191348656
0.7902497226257753


In [34]:
# Importance rank for first model in cross val models
imprtc_df = pd.DataFrame()
imprtc_df['feature'] = sample_train_X.columns   
imprtc_df['importance'] = models[0].feature_importances_
imprtc_df.sort_values('importance', ascending=False, inplace= True)
print(imprtc_df)
print(sample_train_X.columns)

                          feature  importance
0                     building_id        3554
1                           meter         944
71                           hour         701
44      air_temperature_min_lag72         238
29     precip_depth_1_hr_std_lag3         229
43      air_temperature_max_lag72         225
42     air_temperature_mean_lag72         224
5                 log_square_feet         216
52      dew_temperature_min_lag72         183
50     dew_temperature_mean_lag72         147
59   sea_level_pressure_max_lag72         140
51      dew_temperature_max_lag72         139
62      wind_direction_mean_lag72         123
60   sea_level_pressure_min_lag72         123
65       wind_direction_std_lag72         111
14      air_temperature_mean_lag3         108
2                         site_id         104
66          wind_speed_mean_lag72         102
46      cloud_coverage_mean_lag72         102
54   precip_depth_1_hr_mean_lag72          94
49       cloud_coverage_std_lag72 

In [35]:
# %%time
# ## Single fit single model

# gbm = LGBMRegressor(**gbm_params)
# f_train_X, f_train_y = getInFoldXY(train.index)
# gbm.fit(f_train_X, f_train_y)

In [36]:
# Generate test_X
test_X = x_pipes.transform(
    test
        .merge(building_pipes.transform(building), on='building_id', how='left').drop(['row_id'], axis=1)
        .merge(weather_pipes.transform(weather_test), on=['site_id', 'timestamp'], how='left')
    )

print(test_X.sample(n=20,  random_state=42))
print(test_X.shape)

         building_id meter site_id                    primary_use  \
3573457          173     0       2                      Education   
8315486          222     1       2  Entertainment/public assembly   
40305643        1354     2      15                      Education   
16083617         712     0       5                      Education   
37204119        1344     2      15                      Education   
32144852        1119     1      13                         Office   
5105044          249     0       2  Entertainment/public assembly   
36982844        1303     1      14                     Healthcare   
20487823         945     2       9                         Office   
8404196          217     1       2                      Education   
6889602          241     0       2  Entertainment/public assembly   
16963616         784     0       6                      Education   
39666699        1381     2      15                         Office   
26802058        1179     1      13

[20 rows x 72 columns]
(41697600, 72)
